In [ ]:
import os, json, torch, time, sys
import polyfempy as pf
import torch.optim as optim

sys.path.append("../src/")
from functionTarget import FunctionTargetLoss

torch.set_default_dtype(torch.float64)

In [ ]:

class Simulate(torch.autograd.Function):

    @staticmethod
    def forward(ctx, solver, lam, mu):
        # Update solver setup
        solver.set_per_element_material(lam.detach().numpy(), mu.detach().numpy())

        # Enable caching intermediate variables in the simulation, which will be used for solve_adjoint
        solver.set_cache_level(pf.CacheLevel.Derivatives)
        # Run simulation
        solver.solve()
        # Collect transient simulation solutions
        sol = torch.tensor(solver.get_solutions())
        # Cache solver for backward gradient propagation
        ctx.solver = solver
        return sol

    @staticmethod
    @torch.autograd.function.once_differentiable
    def backward(ctx, grad_output):
        # solve_adjoint only needs to be called once per solver, independent of number of types of optimization variables
        ctx.solver.solve_adjoint(grad_output)
        # Compute initial derivatives
        grads = torch.tensor(pf.elastic_material_derivative(ctx.solver))
        return None, grads[0, :], grads[1, :]


In [ ]:
log_level = 3 # warning

def create_solver(args):
    solver = pf.Solver()
    solver.set_settings(json.dumps(args), True)
    solver.set_log_level(log_level)
    solver.load_mesh_from_settings()
    solver.build_basis()
    return solver

In [ ]:
root = "."
with open(root + "/run.json", "r") as f:
    config = json.load(f)
    config["root_path"] = root + "/run.json"

# Simulation
solver = create_solver(config)

lam = (torch.ones((solver.mesh().n_elements()), dtype=float) * 0.01458333)
mu = (torch.ones((solver.mesh().n_elements()), dtype=float) * 0.02916667)
param = torch.vstack((lam[None, :], mu[None, :])).requires_grad_(True)

def loss(param):
    solution = Simulate.apply(solver, param[0, :], param[1, :])

    return FunctionTargetLoss.apply(solver, solution, param, "(y+0.3-0.7*sin(x))^2", ["2*(0.7*sin(x)-0.3-y)*0.7*cos(x)", "2*(y+0.3-0.7*sin(x))"], [2]) + \
    FunctionTargetLoss.apply(solver, solution, param, "(y-0.3-0.7*sin(x))^2", ["2*(0.7*sin(x)+0.3-y)*0.7*cos(x)", "2*(y-0.3-0.7*sin(x))"], [4])

In [ ]:
def verify_grad(input):
    param = input.clone().detach().requires_grad_(True)
    theta = torch.randn_like(param)
    l = loss(param)
    l.backward()
    grad = param.grad
    t = 1e-8
    with torch.no_grad():
        analytic = torch.tensordot(grad, theta)
        f1 = loss(param + theta * t)
        f2 = loss(param - theta * t)
        fd = (f1 - f2) / (2 * t)
        print(f'grad {analytic}, fd {fd} {(f1 - l) / t} {(l - f2) / t}, relative err {abs(analytic - fd) / abs(analytic):.3e}')
        print(f'f(x+dx)={f1}, f(x)={l.detach()}, f(x-dx)={f2}')
        assert(abs(analytic - fd) <= 1e-5 * abs(analytic))

verify_grad(param)

In [ ]:
def save_to_file(x, iter, out_dir):
    sys.stdout.flush()
    outpath = os.path.join(out_dir, "iter_" + str(iter) + ".vtu")
    print("Save to", outpath)
    solver.export_vtu(outpath, solver.get_solutions())

In [ ]:
optimizer = optim.LBFGS([param], lr=1e-1, line_search_fn='strong_wolfe')

# optimization configuration
out_dir = "./opt"
if os.path.exists(out_dir):
    os.system("rm -r " + out_dir)
os.mkdir(out_dir)

# run simulation and compute loss
def closure():
    optimizer.zero_grad()
    l = loss(param)
    l.backward()
    return l

start_time = time.time()
out_dir = os.path.join(os.getcwd(), "opt")
if os.path.exists(out_dir):
    os.system("rm -r " + out_dir)
os.mkdir(out_dir)
for iter in range(15):
    last_loss = closure()
    print(f'Step {iter}: energy {last_loss:.4e}, grad norm {torch.linalg.norm(param.grad):.4e}, total time {time.time() - start_time:.2f} sec')
    save_to_file(param, iter, out_dir)

    # Let optimizer take a step
    optimizer.step(closure)

print("Optimization finished!")